# Calculate Cloud Fraction Statistics

Here, we calculate average area fraction per cloud type and save the result in a file.

In [1]:
%matplotlib inline

import numpy as np
import glob

import pylab as pl
import xarray as xr
import datetime

In [2]:
import nawdex_analysis.analysis.averaging
import nawdex_analysis.io.tools


from tropy.standard_config import local_data_path
from tropy.l15_msevi.msevi import MSevi

import tropy.analysis_tools.grid_and_interpolation as gi

## Make a Function for Averaging

In [3]:

def calculate_ave_afrac( dset, itime, factor = -1, **kwargs ):
       
    '''
    Calculate Cloud-radiative effect (CRE) for different cloud types.

    
    Parameters
    ----------
    dset : xarray Dataset
        set containing CT and mask & area information

    itime : int
       time index of data fields ('swf_net' and 'lwf') in radname

    factor : float
       factor used to convert direction conversion (outward vs. inward)

    filepart : str, optional, default = '-scaled'
       part in the file that gives information about scaling of clear-sky fields
       either '-scaled' or '-not_scaled'


    Returns
    --------
    outset : xarray Dataset
       dataset containing average longwave, short-wave CRE and area fractions
       depending on cloud type
    '''

    
    # prepare analysis array
    m = dset['mask'].data

    ct_field = dset['CT'].isel( time = itime ).data
    a_field = dset['a'].data
    
    time = dset['time'].isel( time = itime )

    a = a_field[m]
    ct = ct_field[m]

    
    
    # calculate area-weighted CRE average
    ctbins =  np.arange(0,22)
    afrac = nawdex_analysis.analysis.averaging.area_fractions( a, ct, ctbins ) * 100.
    
    # rewrite data into xarray
    ct_map = [2, 6, 8, 10, 12, 14, 15, 16, 17, 18, 19]

    ct_labels = [ 'clear_ocean',  'very low', 'low', 'middle', 'high opaque', 
                                 'very high opaque', 'semi. thin', 'semi. meanly thick', 
                                 'semi. thick', 'semi. above', 'fractional'
                             ] 
    
    outset = xr.Dataset({'ct' : ('ct', ct_labels, {}),
                     'time' : ('time', [time.data,], {}),
                     'afrac' : (('time','ct'), np.array( [afrac,] )[:,ct_map], 
                                  {'units' : '%', 'longname':'relative area fractions per cloud type'})})

    return outset


## CT Data Input

### Make a Fileist

In [4]:
#old path
fdir = '%s/icon/hdcp2_atlantic_stochconv_main_experiments' % local_data_path

In [5]:
# new path
fdir = '%s/icon/stoch_exp_v2019-10-18' % local_data_path
fdir = '%s/icon/stoch_exp_v2020-02-13' % local_data_path


In [6]:
flist = glob.glob( '%s/nwcsaf*-hdcp2_atlantic*[!0]??.nc' % fdir)
print( len(flist ))

for i, fname in enumerate( flist ):
    print(i, fname)

5
(0, '/vols/fs1/store/senf/data/icon/stoch_exp_v2020-02-13/nwcsaf_msevi-hdcp2_atlantic-20131220.nc')
(1, '/vols/fs1/store/senf/data/icon/stoch_exp_v2020-02-13/nwcsaf_synsat-hdcp2_atlantic_stochconv_noconvprec_notundepth.nc')
(2, '/vols/fs1/store/senf/data/icon/stoch_exp_v2020-02-13/nwcsaf_synsat-hdcp2_atlantic_stochconv_vervel_-00.nc')
(3, '/vols/fs1/store/senf/data/icon/stoch_exp_v2020-02-13/nwcsaf_synsat-hdcp2_atlantic_detconv.nc')
(4, '/vols/fs1/store/senf/data/icon/stoch_exp_v2020-02-13/nwcsaf_synsat-hdcp2_atlantic_noconv.nc')


In [7]:
#i = 0
#i = 1
i = 2
#i = 3
# i = 4
#i = 5
#i = 6

In [8]:
fname = flist[i]
fname

'/vols/fs1/store/senf/data/icon/stoch_exp_v2020-02-13/nwcsaf_synsat-hdcp2_atlantic_stochconv_vervel_-00.nc'

### Open CT File

In [9]:
dset = xr.open_dataset( fname)

### Convert Time

In [10]:
time = nawdex_analysis.io.tools.convert_timevec( dset.time.data )

In [11]:
dset['time'] = time

In [12]:
dset

<xarray.Dataset>
Dimensions:        (cols: 2050, ndim: 2, rows: 900, time: 23)
Coordinates:
    lat            (rows, cols) float64 ...
    lon            (rows, cols) float64 ...
  * time           (time) datetime64[ns] 2013-12-20T02:00:00 ... 2013-12-21
Dimensions without coordinates: cols, ndim, rows
Data variables:
    msevi_region   (ndim, ndim) int64 ...
    nwcsaf_region  (ndim, ndim) int64 ...
    CTTH_HEIGHT    (time, rows, cols) float32 ...
    CMa            (time, rows, cols) float32 ...
    CT             (time, rows, cols) float32 ...
Attributes:
    description:  Based on synthetic Meteosat-SEVIRI images from the Prime Se...
    title:        NWCSAF cloud products
    institution:  Leibniz Institute for Tropospheric Research
    author:       Fabian Senf (senf@tropos.de)

### Get Mask

In [13]:
maskname = '%s/landseamask.nc' % fdir
lsm = xr.open_dataset( maskname )

In [14]:
lsm

<xarray.Dataset>
Dimensions:  (col: 2050, row: 900)
Coordinates:
    lon      (row, col) float32 ...
    lat      (row, col) float32 ...
Dimensions without coordinates: col, row
Data variables:
    lsm      (row, col) bool ...

In [15]:
dset['mask'] = lsm['lsm']

## Get Gridbox Areas

In [16]:
x, y = gi.ll2xyc( dset['lon'].data, dset['lat'].data )

In [17]:
a = gi.simple_pixel_area(x, y, xy = True)

/vols/fs1/store/senf/.conda/python27mod/lib/python2.7/site-packages/tropy/analysis_tools/grid_and_interpolation.py:990: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  if not np.rank(lon) == 2 and not np.rank(lat) == 2:


In [18]:
dset['a'] = xr.DataArray( a, dims = ('row', 'col'))

In [19]:
ntimes = dset.dims['time']

## Run Calculations

In [20]:
afrac = []
for i in range( ntimes ):
    afrac +=[ calculate_ave_afrac( dset, i ), ]
afrac = xr.merge(afrac)
    

In [21]:

oname = fname.replace('nwcsaf', 'afrac-stats')
afrac.to_netcdf( oname )
